In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import os

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
import torch

model_path = '../Model/sentiment_bert'

tokenizer = BertTokenizer.from_pretrained(model_path)
config = BertConfig.from_pretrained(model_path, output_hidden_states=True)
bert_model = BertForSequenceClassification.from_pretrained(model_path, config=config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
bert_model.eval()

print("Loaded fine-tuned BERT from:", model_path)

Loaded fine-tuned BERT from: ../Model/sentiment_bert


In [4]:
train_data = pd.read_csv("../Dataset/train_preprocessed.csv")
texts = train_data["Phrase"].astype(str).tolist()
labels = train_data["Sentiment"].tolist()

In [5]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        encoded = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return (
            encoded['input_ids'].squeeze(),
            encoded['attention_mask'].squeeze(),
            torch.tensor(self.labels[idx], dtype=torch.long)
        )

In [6]:
dataset = SentimentDataset(texts, labels, tokenizer)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [7]:
class PolicyNetwork(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=128, output_dim=5):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

policy_net = PolicyNetwork().to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(policy_net.parameters(), lr=1e-4)
epochs = 5

for epoch in range(epochs):
    total_loss = 0
    all_preds = []
    all_labels = []

    policy_net.train()
    for input_ids, attention_mask, labels in tqdm(train_loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            cls_embeddings = outputs.hidden_states[-1][:, 0, :]  # [CLS] token

        logits = policy_net(cls_embeddings)
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"[Supervised Epoch {epoch+1}] Loss: {total_loss:.4f} | Accuracy: {acc:.4f}")

100%|██████████| 5853/5853 [12:50<00:00,  7.60it/s]


[Supervised Epoch 1] Loss: 4095.1990 | Accuracy: 0.7212


100%|██████████| 5853/5853 [12:53<00:00,  7.57it/s]


[Supervised Epoch 2] Loss: 4031.4366 | Accuracy: 0.7240


100%|██████████| 5853/5853 [12:48<00:00,  7.61it/s]


[Supervised Epoch 3] Loss: 4015.8397 | Accuracy: 0.7259


100%|██████████| 5853/5853 [12:47<00:00,  7.62it/s]


[Supervised Epoch 4] Loss: 4006.8636 | Accuracy: 0.7258


100%|██████████| 5853/5853 [12:48<00:00,  7.62it/s]


[Supervised Epoch 5] Loss: 4000.5843 | Accuracy: 0.7256


In [9]:
os.makedirs("../Model", exist_ok=True)
torch.save(policy_net.state_dict(), "../Model/policy_net_supervised.pt")
print("Saved policy_net_supervised.pt")

Saved policy_net_supervised.pt
